# Limpieza y procesamiento de datos 

### Importar librerías
Aquí importamos las librerías que vamos a usar en toda la limpieza de datos. La mayoría de estos ya estan instalados en python (pandas, numpy, matplotlib, requests) sin embargo el único que puede que no venga instalado es GeoPandas. Para instalar geopandas consulta [este link](https://geopandas.org/en/stable/getting_started/install.html).

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import re

# esto es solo para evitar que salga una advertencia de pandas (advierte sobre reescribir tablas)
pd.options.mode.chained_assignment = None  # default='warn'

## Limpieza de datos
----
En esta sección se realiza todo el proceso de importación, limpieza y exploración de datos. El propósito es justificar las decisiones al momento de limpiar los datos.

### Datos geográficos

Esta base de datos fue descargada desde la base de [datos cartográficos](https://cartografia.ine.mx/sige8/productosCartograficos/base) del Instituto Nacional Electoral, el fin de esta base es obtener las coordenadas de cada sección, municipio por estado.

In [2]:
## Datos de municipios
mapa = gpd.read_file('datos/cartografia/MUNICIPIO.shp')
mapa = mapa.to_crs(epsg=4326)
mapa.head(3)

,ID,ENTIDAD,MUNICIPIO,NOMBRE,CONTROL,GEOMETRY1_,geometry
0,1835,31,1,ABALA,0.0,None,"POLYGON ((-89.67953 20.73725, -89.67649 20.734..."
1,695,5,1,ABASOLO,0.0,None,"POLYGON ((-101.40708 27.21395, -101.39015 27.2..."
2,2253,11,1,ABASOLO,0.0,None,"POLYGON ((-101.5777 20.32635, -101.5784 20.326..."


In [3]:
mapa.columns = map(str.lower, mapa.columns)
mapa.drop(['control', 'geometry1_', 'id'], axis = 1, inplace=True)
mapa.head(3)

,entidad,municipio,nombre,geometry
0,31,1,ABALA,"POLYGON ((-89.67953 20.73725, -89.67649 20.734..."
1,5,1,ABASOLO,"POLYGON ((-101.40708 27.21395, -101.39015 27.2..."
2,11,1,ABASOLO,"POLYGON ((-101.5777 20.32635, -101.5784 20.326..."


In [4]:
## Datos de Entidades
mapa_E = gpd.read_file('datos/cartografia/ENTIDAD.shp') ## Aquí ingresa el filepath
mapa_E = mapa_E.to_crs(epsg=4326)
mapa_E.columns = map(str.lower, mapa_E.columns)
mapa_E.drop(['control', 'geometry1_', 'crc', 'circunscri', 'id'], axis = 1, inplace=True)
mapa_E.head(3)

,entidad,nombre,geometry
0,1,AGUASCALIENTES,"POLYGON ((-101.86092 22.02551, -101.85872 22.0..."
1,2,BAJA CALIFORNIA,"MULTIPOLYGON (((-116.81053 31.81312, -116.8100..."
2,3,BAJA CALIFORNIA SUR,"MULTIPOLYGON (((-112.08233 25.34398, -112.0822..."


In [5]:
## Datos de Secciones Electorales
mapa_S = gpd.read_file('datos/cartografia/SECCION.shp') ## Aquí ingresa el filepath
mapa_S = mapa_S.to_crs(epsg=4326)
mapa_S.columns = map(str.lower, mapa_S.columns)
mapa_S.drop(['control', 'geometry1_', 'distrito_f', 'distrito_l', 'tipo', 'id'], axis = 1, inplace=True)
mapa_S.head(3)

,entidad,municipio,seccion,geometry
0,4,1,1,"POLYGON ((-90.49402 19.87148, -90.49506 19.870..."
1,24,1,1,"POLYGON ((-101.15731 22.40989, -101.15711 22.4..."
2,3,1,1,"POLYGON ((-111.68825 25.04847, -111.6839 25.04..."


### Datos del DENUE

En esta sección importamos datos del Directorio Estadístico Nacional de Unidades Económicas usando el API del INEGI, el fin de usar un API fue para hacer mas eficiente el proceso de busqueda de unidades económicas de nuestro interes. Más información del API haz [click aquí]('https://www.inegi.org.mx/servicios/api_denue.html').

In [6]:
## Llave de acceso al api
denue_api_token = '5fba9638-bc43-4847-9ade-35d4a7b2b9cf'
## Este url devuelve todas las farmacias registradas en toda la républica Mexicana
url = f'https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarEntidad/farmacias/00/0/80000/{denue_api_token}'
## Importar los datos y convertirlos en tablas
datos = requests.get(url).json()
farmacias_df = pd.DataFrame(datos)
farmacias_df.head()


,CLEE,Id,Nombre,Razon_social,Clase_actividad,Estrato,Tipo_vialidad,Calle,Num_Exterior,Num_Interior,...,Ubicacion,Telefono,Correo_e,Sitio_internet,Tipo,Longitud,Latitud,tipo_corredor_industrial,nom_corredor_industrial,numero_local
0,15039464111005171000000000U9,8035570,@MERY PHARMA,,Farmacias sin minisúper,0 a 5 personas,CALLE,JACARANDAS,,,...,"IXTAPALUCA, Ixtapaluca, MÉXICO ...",5558262668,,,Fijo,-98.90520467,19.30340293,,,
1,15109464111005861000000000U6,8361679,+ FARMA,,Farmacias sin minisúper,0 a 5 personas,AVENIDA,RANCHO SANTA MARIA,,,...,"FUENTES DEL VALLE, Tultitlán, MÉXICO ...",,,,Fijo,-99.13373695,19.63500170,,,
2,22014464111003821000000000S5,3582548,01 + AHORRO FARMACIAS,,Farmacias sin minisúper,0 a 5 personas,CALLE,EJE NORTE,425,,...,"SANTIAGO DE QUERÉTARO, Querétaro, QUERÉTARO ...",,FARMATLALOC@YAHOO.MX,,Fijo,-100.41587167,20.62688890,,,
3,16034464111001171000000000S3,8953165,100 % FARMA EXPRESS,100 % LA FARMA EXPRESS LA FUENTE,Farmacias sin minisúper,0 a 5 personas,CALLE,VICENTE GUERRERO,153,,...,"CIUDAD HIDALGO, Hidalgo, MICHOACÁN DE OCAMPO ...",7861426289,,,Fijo,-100.56290805,19.69435866,,,
4,15076464112000162000000000S3,7914296,100 CIEN POR CIENTO FARMA,FENAHAME SA DE CV,Farmacias con minisúper,6 a 10 personas,AVENIDA,BENITO JUAREZ,,,...,"SAN MATEO ATENCO, San Mateo Atenco, MÉXICO ...",,SANMATEOSINFARMA@GMAIL.COM,,Fijo,-99.52955867,19.26460193,,,


Viendo las columnas que tiene esta tabla, hay una que nos interesa mucho *clase_actividad* ya que podemos usar esta columna para realmente observar los establecimientos que obtuvimos son farmacias.

In [7]:
## Observamos los tipos de actividades que hay en nuestras observaciones
farmacias_df.Clase_actividad.unique()

array(['Farmacias sin minisúper', 'Farmacias con minisúper',
       'Comercio al por menor de productos naturistas, medicamentos homeopáticos y de complementos alimenticios',
       'Comercio al por menor en tiendas de abarrotes, ultramarinos y misceláneas',
       'Comercio al por mayor de abarrotes',
       'Comercio al por menor en minisupers',
       'Comercio al por menor de leche, otros productos lácteos y embutidos',
       'Servicios de preparación de otros alimentos para consumo inmediato',
       'Guarderías del sector privado',
       'Comercio al por mayor de medicamentos veterinarios y alimentos para animales, excepto mascotas',
       'Asociaciones y organizaciones civiles',
       'Clínicas de consultorios médicos del sector privado',
       'Comercio al por menor de mascotas',
       'Comercio al por mayor de productos farmacéuticos',
       'Laboratorios médicos y de diagnóstico del sector privado',
       'Consultorios de medicina general del sector privado',
       '

Como se puede observar, hay negocios en esta tabla que no pertenecen a la categoría de **farmacia** por lo que debemos de filtrar estos establecimientos

In [8]:
filtro = r'\bFarmacias\b'
filtrado = farmacias_df.Clase_actividad.str.match(filtro)
farmacias_df = farmacias_df[filtrado]

In [9]:
## Imprimir la infromación general de las columnas
farmacias_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65105 entries, 0 to 73599
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CLEE                      65105 non-null  object
 1   Id                        65105 non-null  object
 2   Nombre                    65105 non-null  object
 3   Razon_social              65105 non-null  object
 4   Clase_actividad           65105 non-null  object
 5   Estrato                   65105 non-null  object
 6   Tipo_vialidad             65105 non-null  object
 7   Calle                     65105 non-null  object
 8   Num_Exterior              65105 non-null  object
 9   Num_Interior              65105 non-null  object
 10  Colonia                   65105 non-null  object
 11  CP                        65105 non-null  object
 12  Ubicacion                 65105 non-null  object
 13  Telefono                  65105 non-null  object
 14  Correo_e                  6

Como se puede observar, todas las variables son del tipo *object*, sin embargo, para poder gráficar las farmacias en el mapa, convertimos la Latitud y Longitud en variables númericas. Igualmente descartamos variables que no consideramos que son relevantes para este estudio. 

In [10]:
## Reescribir en minúsculas los nombres de las columnas
farmacias_df.columns = map(str.lower, farmacias_df.columns)
## Eliminar las columnas que no nos son relevantes
farmacias_df = farmacias_df.drop(['tipo_corredor_industrial', 'nom_corredor_industrial', 'numero_local', 'sitio_internet', 'correo_e', 'telefono', 'num_interior', 'clee', 'id'], axis = 1)
## Convertir los valores tipo string en númericos
farmacias_df['longitud'] = farmacias_df.longitud.apply(lambda x : float(x))
farmacias_df['latitud'] = farmacias_df.latitud.apply(lambda x : float(x))

## Convertir las coordenadas de longitud y latitud en un objeto geométrico (esta acción tendrá relevancia más adelante)
farmacias_df = gpd.GeoDataFrame(farmacias_df, geometry=gpd.points_from_xy(farmacias_df.longitud, farmacias_df.latitud), crs="EPSG:4326")
farmacias_df.head(1)

,nombre,razon_social,clase_actividad,estrato,tipo_vialidad,calle,num_exterior,colonia,cp,ubicacion,tipo,longitud,latitud,geometry
0,@MERY PHARMA,,Farmacias sin minisúper,0 a 5 personas,CALLE,JACARANDAS,,ALFREDO DEL MAZO,56577,"IXTAPALUCA, Ixtapaluca, MÉXICO ...",Fijo,-98.905205,19.303403,POINT (-98.9052 19.3034)


Para finalizar vamos a hacer la distinción respecto a si la farmacia de cada observación es parte del grupo ANTAD.
Para este realizar este filtrado nos basamos en la razón social de cada negocio perteneciente a dicho grupo. La tabla que usamos de referencia esta en el [siguiente link](https://antad.net/wp-content/uploads/2022/11/estructura-asociativa_18nov22.pdf).

In [11]:
## Filtrar 
patron = r"(COMERCIALIZADORA FARMACEUTICA DE CHIAPAS|FARMACIAS DEL AHORRO|FARMACIAS BENAVIDES|FARMACIA GUADALAJARA|KLYN'S FARMACIAS|DROGUERÍA Y FARMACIA EL FÉNIX|FENIX)"
filtrado = farmacias_df.razon_social.str.match(patron)
farmacias_df['antad'] = filtrado
farmacias_df.head(3)

,nombre,razon_social,clase_actividad,estrato,tipo_vialidad,calle,num_exterior,colonia,cp,ubicacion,tipo,longitud,latitud,geometry,antad
0,@MERY PHARMA,,Farmacias sin minisúper,0 a 5 personas,CALLE,JACARANDAS,,ALFREDO DEL MAZO,56577,"IXTAPALUCA, Ixtapaluca, MÉXICO ...",Fijo,-98.905205,19.303403,POINT (-98.9052 19.3034),False
1,+ FARMA,,Farmacias sin minisúper,0 a 5 personas,AVENIDA,RANCHO SANTA MARIA,,VILLAS DE SAN JOSE,54910,"FUENTES DEL VALLE, Tultitlán, MÉXICO ...",Fijo,-99.133737,19.635002,POINT (-99.13374 19.635),False
2,01 + AHORRO FARMACIAS,,Farmacias sin minisúper,0 a 5 personas,CALLE,EJE NORTE,425,FRACC VILLAS TLALOC,76121,"SANTIAGO DE QUERÉTARO, Querétaro, QUERÉTARO ...",Fijo,-100.415872,20.626889,POINT (-100.41587 20.62689),False


In [12]:
farmacias_df.to_csv('farmacias.csv', index= False)

### Datos del Censo 2020

En esta sección vamos a descargar, explorar y limpiar los datos del Censo 2020, para consultarla fuente de esta base haz [click aquí](https://www.inegi.org.mx/programas/ccpv/2020/#Datos_abiertos). 

In [13]:
## Importar los datos
raw_data = pd.read_csv('datos/censo2020/conjunto_de_datos/conjunto_de_datos_iter_00CSV20.csv')
                        ## Aquí solo hay que ingresar el csv del censo2020
dic_df = pd.read_csv('datos/censo2020/diccionario_datos/diccionario_datos_iter_00CSV20.csv', skiprows= 4)
dic_df.drop(dic_df.columns[-4:],axis=1,inplace=True)
raw_data.head()

/var/folders/96/mz_mcrd16ggfhtvkbgqt5m0h0000gn/T/ipykernel_99994/2257295073.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('datos/censo2020/conjunto_de_datos/conjunto_de_datos_iter_00CSV20.csv')


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


Podemos observar que las columnas *LONGITUD*, *LATITUD* y *ALTITUD* contienen valores `NaN`, esto se significa que las observaciones que contienen estos valores son indicadores generalizados de la ragión, mas no nos dicen los valores especificos por localidad o municipio. Para limpiar estos datos, solo hay que eliminar las observaciones con los valores `NaN` en las columnas *LONGITUD*, *LATITUD* y *ALTITUD*. 

Igualmente por fines prácticos, reescribimos cambiamos los nombres de las columnas en minúsculas

In [14]:
## Reescribir los nombres de las columnas en minúsuclas
raw_data.columns = map(str.lower, raw_data.columns)
## Retirar las observaciones con valores NaN de la tabla
datos = raw_data.dropna(subset=['longitud', 'latitud', 'altitud']) 
datos.head()

,entidad,nom_ent,mun,nom_mun,loc,nom_loc,longitud,latitud,altitud,pobtot,...,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,tamloc
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102°17'45.768"" W","21°52'47.362"" N",1878.0,863893,...,232793,169675,123670,77719,53589,2995,5984,63661,595,13
8,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102°22'24.710"" W","21°52'18.749"" N",1902.0,5,...,*,*,*,*,*,*,*,*,*,1
9,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102°21'25.639"" W","21°53'01.522"" N",1861.0,41,...,11,4,5,2,1,0,1,6,0,1
10,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102°21'26.261"" W","21°46'48.650"" N",1861.0,8,...,*,*,*,*,*,*,*,*,*,1
11,1,Aguascalientes,1,Aguascalientes,104,Ardillas de Abajo (Las Ardillas),"102°11'30.914"" W","21°56'42.243"" N",1989.0,1,...,*,*,*,*,*,*,*,*,*,1


Si observamos las columnas *longitud* y *latitud*, veremos que las coordenas asimilan una formato de coordenas polares, sin embargo no es posible poder graficar con estos datos, además son valores tipo *string*. Para solucionar esto, creamos una función para tranformar estas coordenadas en valores númericos.

In [15]:
## Función para convertir las coordenadas en formato string a valores númericos (float)
def conversion(original):
    dir = {'N':1, 'S':-1, 'E': 1, 'W':-1}
    nuevo = original.replace(u'°',' ').replace('\'',' ').replace('"',' ')
    nuevo = nuevo.split()
    n_dir = nuevo.pop()
    nuevo.extend([0,0,0])
    return (float(nuevo[0])+float(nuevo[1])/60.0+float(nuevo[2])/3600.0) * dir[n_dir]

## Aplicar la función en las columnas longitud y latitud
datos.longitud = datos.longitud.apply(lambda x : conversion(x))
datos.latitud = datos.latitud.apply(lambda x : conversion(x))

## En esta línea cambiamos los * por valores NaN 
datos.replace('*', np.nan, inplace = True)


Una vez solucionado este problema solo hay que confirmar que las observaciones de los indicadores sean valores númericos. Sin embargo sí imprimimos la infromación general de la tabla observaremos que hay 278 columnas con valores tipo *object*. Lo que significa que los valores no son númericos.

In [16]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189432 entries, 7 to 195660
Columns: 286 entries, entidad to tamloc
dtypes: float64(2), int64(6), object(278)
memory usage: 414.8+ MB


Para solucionar esto solo hay que convertir los valores tipo *string* en valores númericos.

In [17]:
## función para convertir los datos string a valores númericos
def convertir_str_a_num(df) : 
    indicadores = df.columns
    for i in range(10, datos.shape[1]) : 
        df[indicadores[i]] = pd.to_numeric(df[indicadores[i]], errors='coerce')

    return df

## la nueva tabla con los datos númericos
censo_df = convertir_str_a_num(datos)

Como podemos observar ya solo tenemos 4 datos tipo *object*, los cuales deben de ser los nombres de los estados y localidades del censo.

In [18]:
censo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189432 entries, 7 to 195660
Columns: 286 entries, entidad to tamloc
dtypes: float64(275), int64(7), object(4)
memory usage: 414.8+ MB


## Procesamiento de datos
----
En esta sección las bases de datos que ya están limpias van a ser manipuladas para faciliar el proceso de análisis de datos. 

El objetivo en este proyecto es:
1. Con la base de datos de farmacias que tenemos y la tabla de datos cartográficos, contamos cuantas farmacias hay por estado, municipio y sección electoral
2. Un vez que tenemos el número farmacias por municipio, ahora hay que anexar estos datos con los datos del censo par así tener una sola tabla con los datos de nuestro interés

### Conteo de farmacias por municipio, sección y entidad

En el siguiente código creamos una función para contar cuantas farmacias hay por estado, lo que hace la función es agarrar las coordenadas de cada farmacia de la columna *geometry* en `farmacias_df` y checar si ese punto está dentro de la área de cada municipio, dicha área es representada por un objeto tipo polígono en la columna *geometry* de `mapa`.


In [19]:
## Función para contar farmacias, devuelve el mapa con el número de farmacias por municipio
def contador_de_unidades(map, unidades):
    contador_antad = []
    contador_no_antad = []
## el pepe
    for geo_point, antad in zip(unidades.geometry, unidades.antad) : 
        resultado = map.contains(geo_point)
        if antad : 
            contador_antad.append(resultado)
        else : 
            contador_no_antad.append(resultado)

    contador_antad_df = pd.DataFrame(contador_antad).transpose()
    contador_no_antad_df = pd.DataFrame(contador_no_antad).transpose()

    map['far_antad'] = contador_antad_df.sum(axis=1)
    map['far_no_antad'] = contador_no_antad_df.sum(axis=1)
    map['far_total'] = map['far_antad'] + map['far_no_antad']
    return map

mapa_municipios_far = contador_de_unidades(mapa, farmacias_df)
mapa_entidades_far = contador_de_unidades(mapa_E, farmacias_df)
mapa_secciones_far = contador_de_unidades(mapa_S, farmacias_df)

Como podemos observar en la tabla `mapa_municipios_far` ahora hay una nueva columna que contiene el número de farmacias por municipio.

In [20]:
mapa_municipios_far.head(3)

,entidad,municipio,nombre,geometry,far_antad,far_no_antad,far_total
0,31,1,ABALA,"POLYGON ((-89.67953 20.73725, -89.67649 20.734...",0,0,0
1,5,1,ABASOLO,"POLYGON ((-101.40708 27.21395, -101.39015 27.2...",0,0,0
2,11,1,ABASOLO,"POLYGON ((-101.5777 20.32635, -101.5784 20.326...",2,26,28


### Conectar las tablas

En esta sección juntaremos el número de farmacias por municipio en `mapa_municipios_far` con los datos del censo en `censo_df`. Adicionalmente se va a incluir los polígonos de las áreas geográficas que representan los municipios para faciliatr el proceso de graficado al momento de visualizar datos.

Como primer paso vamos agrupar los datos de `censo_df` por municipio:

In [21]:
## Definir el método sobre el cual haremos la agrupación
cols = censo_df.columns[9:censo_df.shape[1]]
dic_con_metodos = { ind : 'sum' for ind in cols}

## Hacer la agrupación de los datos
censo_municipios = censo_df.groupby(['entidad', 'mun', 'nom_ent', 'nom_mun'], as_index=False).agg(dic_con_metodos)
censo_entidades = censo_df.groupby(['entidad','nom_ent'], as_index=False).agg(dic_con_metodos)

censo_municipios.head(3)

,entidad,mun,nom_ent,nom_mun,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,...,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,tamloc
0,1,1,Aguascalientes,Aguascalientes,948990,486138.0,460305.0,44297.0,21854.0,22443.0,...,251390.0,178518.0,130206.0,80923.0,56106.0,3287.0,7275.0,74004.0,725.0,704
1,1,2,Aguascalientes,Asientos,51536,26097.0,25079.0,3154.0,1630.0,1524.0,...,10592.0,4494.0,3848.0,590.0,551.0,371.0,1439.0,7152.0,176.0,268
2,1,3,Aguascalientes,Calvillo,58250,29584.0,28430.0,3146.0,1576.0,1570.0,...,13602.0,6537.0,4736.0,1380.0,1350.0,436.0,895.0,8006.0,134.0,236


Como la unión que queremos hacer de la tabla `censo_municipios` con `mapa_municipios_far` será sobre la columna *entidad* y *municipio* hay que checar que ambas columnas tengan los mismos nombres. 

In [22]:
mapa_municipios_far.head(1)

,entidad,municipio,nombre,geometry,far_antad,far_no_antad,far_total
0,31,1,ABALA,"POLYGON ((-89.67953 20.73725, -89.67649 20.734...",0,0,0


Como pueden osbervar, las tablas tienen el mismo nombre para la columna de entidades (*entidad*) pero diferen en sus columnas que representan los municipios (una tiene *mun* y la otra tiene *municipio*). Entonces solo hay que renombrar la columna.

In [23]:
## Renombrar la columna 
censo_municipios.rename(columns={'mun' : 'municipio'}, inplace= True)
censo_municipios.head(1)

,entidad,municipio,nom_ent,nom_mun,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,...,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,tamloc
0,1,1,Aguascalientes,Aguascalientes,948990,486138.0,460305.0,44297.0,21854.0,22443.0,...,251390.0,178518.0,130206.0,80923.0,56106.0,3287.0,7275.0,74004.0,725.0,704


Ahora sí, se puede hacer el *join* de las tablas. Al final se puede observar que lel número de farmacias con su área geográfica están incluidas en la tabla `municipios_df`. Solamente hay que checar si por la unión surgieron datos `NaN`.

In [24]:
municipios_df = pd.merge(censo_municipios, mapa_municipios_far, how='outer', on = ['entidad', 'municipio'])
entidades_df  = pd.merge(censo_entidades, mapa_entidades_far, how='outer', on = ['entidad'])
municipios_df.head(2)


,entidad,municipio,nom_ent,nom_mun,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,...,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,tamloc,nombre,geometry,far_antad,far_no_antad,far_total
0,1,1,Aguascalientes,Aguascalientes,948990.0,486138.0,460305.0,44297.0,21854.0,22443.0,...,3287.0,7275.0,74004.0,725.0,704.0,AGUASCALIENTES,"POLYGON ((-102.60076 21.75515, -102.6016 21.76...",79.0,270.0,349.0
1,1,2,Aguascalientes,Asientos,51536.0,26097.0,25079.0,3154.0,1630.0,1524.0,...,371.0,1439.0,7152.0,176.0,268.0,ASIENTOS,"POLYGON ((-101.86092 22.02551, -101.85872 22.0...",0.0,16.0,16.0


Checamos si hay valores `NaN`

In [25]:
print('Checar valores NaN para municipios_df')
print(municipios_df.isna().any(axis=1).value_counts())
print('Checar valores NaN para entidades_df')
print(entidades_df.isna().any(axis=1).value_counts())

Checar valores NaN para municipios_df
False    2466
True       14
Name: count, dtype: int64
Checar valores NaN para entidades_df
False    32
Name: count, dtype: int64


Como podemos observar hay 14 valores `nan` en la tabla municipios_df, es probable que esto haya sucedido por una observación que sí tenía los datos completos en una tabla pero que estaba incompleta en la otra tabla. A continuación checamos las observaciones con valores `NaN`.

In [26]:
municipios_df[municipios_df.isna().any(axis=1)]

,entidad,municipio,nom_ent,nom_mun,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,...,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,tamloc,nombre,geometry,far_antad,far_no_antad,far_total
17,2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SAN FELIPE,"MULTIPOLYGON (((-114.46309 31.10729, -114.4626...",0.0,11.0,11.0
21,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,LOS CABOS,"POLYGON ((-109.68188 23.6498, -109.68055 23.64...",3.0,190.0,193.0
22,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,LORETO,"MULTIPOLYGON (((-111.31017 26.09041, -111.3093...",0.0,4.0,4.0
23,3,8,Baja California Sur,Los Cabos,351111.0,169688.0,179848.0,17923.0,8866.0,9057.0,...,7245.0,3260.0,32749.0,1023.0,466.0,NaN,None,NaN,NaN,NaN
24,3,9,Baja California Sur,Loreto,18052.0,8650.0,9022.0,788.0,398.0,390.0,...,285.0,315.0,2416.0,76.0,134.0,NaN,None,NaN,NaN,NaN
37,4,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SEYBAPLAYA,"POLYGON ((-90.66962 19.71996, -90.66955 19.719...",0.0,5.0,5.0
180,7,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TEOPISCA,"POLYGON ((-92.54795 16.62022, -92.54767 16.620...",1.0,18.0,19.0
210,7,125,Chiapas,Honduras de la Sierra,11650.0,5728.0,5864.0,751.0,375.0,376.0,...,632.0,1361.0,2170.0,545.0,86.0,NaN,None,NaN,NaN,NaN
460,12,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ÑUU SAVI,"POLYGON ((-98.92152 16.99958, -98.91658 16.980...",0.0,0.0,0.0
461,12,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,LAS VIGAS,"POLYGON ((-99.2725 16.64393, -99.27935 16.6452...",0.0,6.0,6.0


Con esta tabla podemos hacer varias conclusiones: 
1. Hay 3 municipios que sí tienen información del censo pero que no tienen datos geográficos: Loreto y Los Cabos en Baja California al igual que Honduras de la Sierra en Chiapas. 
2. Hay 11 municipios en donde no hay datos de haber sido censados pero sí tienen registros de farmacias.

***¿Cómo podemos solucionar esto?***

Si observamos con más detalle podemos que ver que Loreto y Los Cabos si tienen las farmacias y los datos geográficos registrados, solamente están en otra observación. ¿Por qué habrá sucedido esto? Si observamos la columna *mun* de la tabla, podemos ver que Los Cabos y Loreto son los municipios número 8 y 9 respectivamente de Baja Clifornia Sur, pero Baja Clifornia Sur solo tiene 5 municipios!
Entonces solo tenemos que cambiar el 8 y el 9 por un 4 y un 5.


Por otro lado, como no encontramos una razón que justifique la falta de datos en estos municipios lo más propio será retirar las observaciones y considerar esta acción al momento de analizar los datos y hacer conclusiones.

In [27]:
censo_municipios[censo_municipios['entidad'] == 3] = censo_municipios[censo_municipios['entidad'] == 3].replace([8,9], [4,5])
municipios_df.dropna(inplace=True)
municipios_df.isna().any(axis=1).value_counts()

False    2466
Name: count, dtype: int64

Como se puede observar ya no hay valores `NaN`que interrumpan el proceso de análisis de datos.
Finalmente solo vamos a convertir la tabla en un objeto de *Geopandas* para hacer mapas con mayor facilidad.

In [28]:
municipios_df.drop(['nombre'], axis=1, inplace=True)
municipios_df.head(1)

,entidad,municipio,nom_ent,nom_mun,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,...,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,tamloc,geometry,far_antad,far_no_antad,far_total
0,1,1,Aguascalientes,Aguascalientes,948990.0,486138.0,460305.0,44297.0,21854.0,22443.0,...,56106.0,3287.0,7275.0,74004.0,725.0,704.0,"POLYGON ((-102.60076 21.75515, -102.6016 21.76...",79.0,270.0,349.0


In [29]:
## Cabiamos los objetos DataFrame a objetos GeoDataFrame para gráficar con mayor facilidad
municipios_df = gpd.GeoDataFrame(municipios_df)
entidades_df = gpd.GeoDataFrame(entidades_df)

## Exportar los datos limpios
El siguiente código es solo para exportar las tablas que previamente limpiamos y procesamos a una carpeta para poder retomarlo en otro script para analizar la infromación de este.

In [31]:
## Exportar municipios_df
municipios_df.to_file('datos_procesados/municipios')
## Exportar entidades_df
entidades_df.to_file('datos_procesados/entidades')


/var/folders/96/mz_mcrd16ggfhtvkbgqt5m0h0000gn/T/ipykernel_99994/3609546946.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  municipios_df.to_file('datos_procesados/municipios')
/Users/danielvelez/miniforge3/envs/dataton/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'pcdisc_leng' to 'pcdisc_len'
  ogr_write(
/Users/danielvelez/miniforge3/envs/dataton/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'pcdisc_mot2' to 'pcdisc_m_1'
  ogr_write(
/Users/danielvelez/miniforge3/envs/dataton/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'pclim_re_co' to 'pclim_re_c'
  ogr_write(
/Users/danielvelez/miniforge3/envs/dataton/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'pafil_ipriv' to 'pafil_ipri'
  ogr_write(
/Users/danielvelez/miniforg